# Tutorial for GridSearchCV

In [1]:
%load_ext autoreload
%autoreload 2

import torch
import btorch
from btorch import nn
import btorch.nn.functional as F
from btorch.nn import GridSearchCV
import pandas as pd
from tqdm import tqdm

# Create Model

Lets create a simple NN model for binary classification

In [2]:
class Net(nn.Module):
    def __init__(self, hidden_dim):
        super(Net, self).__init__()
        self.l1 = nn.Linear(50,hidden_dim)
        self.l2 = nn.Linear(hidden_dim,1)
    def forward(self,x):
        return self.l2(torch.relu(self.l1(x)))

Now we want to define a scoring function that takes in ``model_ouput`` and ``y_true`` as argument.  
Remember you must define the parameter name as such.  
This scoring function should accept a batched-input and return the **sum** of score of that batch.

In [3]:
def accuarcy(model_output, y_true):
    y_pred = ((torch.sigmoid(model_output))>0.5).int().float()
    out = (y_pred == y_true).float().sum().item()
    return out

Create a dumpy dataset

In [4]:
x = torch.randn(100, 50)
y = torch.rand(100, 1).round()

## Init the Model

In [5]:
# Here we want to search through the ``hidden_dim`` param in Net()
# Also try two learning rate in Adam()
# Note that the dict key name that start with 'lr_s', 'optim_', 'lossfn_' is reversed and cannot be used
param_grid = {'hidden_dim':[20,30,40]}
optim_grid = {'lr':[0.01, 0.1]}
a = GridSearchCV(Net, param_grid, optim_param_grid=optim_grid, scoring=accuarcy)

# Define the lossfn, optimizer, those thing as usual.
# Something different is that you are now passing the Class to them, instead of Class_instance
# For optimizer and lr_scheduler, you must use ``partial`` to wrap it first
# Since we would like to search through the learning_rate, you leave the ``lr`` arg empty in ``partial``
from functools import partial
a._lossfn = nn.nn.BCEWithLogitsLoss
# a._lossfn = nn.nn.BCEWithLogitsLoss() # WRONG
a._optimizer = partial(torch.optim.Adam, betas=(0.9, 0.999))
# a._optimizer = torch.optim.Adam # WRONG
a._lr_scheduler = partial(torch.optim.lr_scheduler.StepLR, step_size=2)
# a._lr_scheduler = torch.optim.lr_scheduler.StepLR # WRONG
a._config['max_epoch'] = 5

## Fit the Model

In [6]:
a.fit(x, y, verbose=0)

## Check cv_results_

In [7]:
pd.DataFrame(a.cv_results_)

,params,split0_train_loss,split0_test_loss,split0_train_score,split0_test_score,split1_train_loss,split1_test_loss,split1_train_score,split1_test_score,split2_train_loss,...,std_train_loss,std_test_loss,rank_train_loss,rank_test_loss,mean_train_score,mean_test_score,std_train_score,std_test_score,rank_train_score,rank_test_score
0,"{'hidden_dim': 20, 'optim_lr': 0.01}",0.448203,0.673029,0.925373,0.363636,0.357867,0.695965,0.940299,0.484848,0.478563,...,0.428211,0.689926,6.0,1.0,0.919870,0.459299,0.919870,0.459299,1.0,3.0
1,"{'hidden_dim': 20, 'optim_lr': 0.1}",0.115072,1.336728,0.970149,0.424242,0.046724,1.899860,1.000000,0.545455,0.229140,...,0.130312,1.619420,3.0,4.0,0.954696,0.509507,0.954696,0.509507,2.0,6.0
2,"{'hidden_dim': 30, 'optim_lr': 0.01}",0.366376,0.694412,0.970149,0.363636,0.358343,0.741321,0.955224,0.484848,0.401218,...,0.375312,0.697648,5.0,2.0,0.954922,0.449495,0.954922,0.449495,3.5,2.0
3,"{'hidden_dim': 30, 'optim_lr': 0.1}",0.050896,2.038369,1.000000,0.575758,0.148949,2.120383,0.940299,0.484848,0.102917,...,0.100921,1.943066,2.0,5.0,0.964948,0.500594,0.964948,0.500594,5.0,5.0
4,"{'hidden_dim': 40, 'optim_lr': 0.01}",0.369939,0.647532,0.955224,0.424242,0.294470,0.808913,0.970149,0.454545,0.368426,...,0.344279,0.722454,4.0,3.0,0.954922,0.439988,0.954922,0.439988,3.5,1.0
5,"{'hidden_dim': 40, 'optim_lr': 0.1}",0.113039,1.791551,0.970149,0.454545,0.053935,2.424563,0.985075,0.454545,0.072590,...,0.079855,1.969020,1.0,6.0,0.985075,0.469697,0.985075,0.469697,6.0,4.0


## Get the best model

In [8]:
a.best_model_

Net(
  (l1): Linear(in_features=50, out_features=40, bias=True)
  (l2): Linear(in_features=40, out_features=1, bias=True)
  (_lossfn): BCEWithLogitsLoss()
)

In [9]:
print(a.best_score_)
print(a.best_loss_)
print(a.best_params_)

0.42424242424242425
0.647532120347023
{'hidden_dim': 40, 'optim_lr': 0.01}


In [10]:
# This is actually overfitting, we test on training set
# But this shows the model works
accuarcy(a(x), y)

78.0

In [11]:
# Try on new dataset, the results should be around 50% as the dataset is random
x_test = torch.randn(100, 50)
y_test = torch.rand(100, 1).round()
accuarcy(a(x_test), y_test)

51.0